# Test CNN-VAE (reduction) with Fully-Connected Net (prediction)

In [ ]:
import os
from os.path import join
import sys
from pathlib import Path

# include app directory into sys.path
parent_dir = Path(os.path.abspath('')).parent
app_dir = join(parent_dir, "app")
if app_dir not in sys.path:
      sys.path.append(app_dir)

import torch as pt
from torch.nn.functional import mse_loss
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import utils.config as config
from utils.helper_funcs import find_target_index_in_dataset
from utils.CNN_VAE import make_VAE_model
from utils.FullyConnected import make_FC_model
from utils.helper_funcs import shift_input_sequence
from utils.DataWindow import DataWindow

plt.rcParams["figure.dpi"] = 180

# use GPU if possible
device = pt.device("cuda") if pt.cuda.is_available() else pt.device("cpu")
print(device)

DATA_PATH = join(parent_dir, "data", "single_flow_cond")
VAE_PATH = join(parent_dir, "output", "VAE", "latent_study", config.VAE_model)
FC_MODEL = "40_256_1"
FC_PATH = join(parent_dir, "output", "VAE_FC", "param_study", "pred_horizon_6")
LATENT_SIZE = config.VAE_latent_size
OUTPUT_PATH = join(parent_dir, "output", "VAE_FC", "param_study")

PRED_HORIZON = 6
TIMESTEP = config.timestep_reconstruction_single
TIMESTEP_dimless = (TIMESTEP * config.U_inf) / (config.c_mean * config.timesteps_per_second)
TIMESTEP = int(TIMESTEP - config.time_steps_per_cond * config.single_flow_cond_train_share) # to receive index of test data subtract number of samples in train data
print(TIMESTEP_dimless)

#### Load data (single flow condition at alpha = 4.00)

In [ ]:
# load datasets
train = pt.load(join(DATA_PATH, "VAE_train.pt"))
test = pt.load(join(DATA_PATH, "VAE_test.pt"))
print(train.max(), train.min())
print(test.max(), test.min())
print(train.shape)
print(test.shape)

coords = pt.load(join(Path(DATA_PATH).parent, "coords_interp.pt"))
xx, yy = coords

#### Load autoencoder and encode data

In [ ]:
# load pre-trained autoencoder model
autoencoder = make_VAE_model(
    n_latent=LATENT_SIZE, 
    device=device)
autoencoder.load(VAE_PATH)
autoencoder.eval()
decoder = autoencoder._decoder

# encode datasets
train_enc = autoencoder.encode_dataset(train)
test_enc = autoencoder.encode_dataset(test)
print(test_enc.shape)

#### Load Fully-Connected Net

In [ ]:
# results from parameter study
INPUT_WIDTH = 40
HIDDEN_SIZE = 256
N_HIDDEN_LAYERS = 1

FC_model = make_FC_model(
    latent_size=LATENT_SIZE,
    input_width=INPUT_WIDTH, 
    hidden_size=HIDDEN_SIZE, 
    n_hidden_layers=N_HIDDEN_LAYERS
)
FC_model.load(join(FC_PATH, FC_MODEL + ".pt"))
FC_model.eval()

#### Scale data, create Data Window and load into Time-Series TensorDataset to create input-target pairs

In [ ]:
latent_scaler = pt.load(join(FC_PATH, "scaler.pt"))
data_window = DataWindow(train=latent_scaler.scale(train_enc), test=latent_scaler.scale(test_enc), input_width=INPUT_WIDTH, pred_horizon=PRED_HORIZON)
_, target_idx = data_window.rolling_window(test_enc.shape[1])
target_idx = target_idx.tolist()
test_enc = data_window.test_dataset

#### Compute autoregressive prediction

In [ ]:
# initialize losses
latent_loss = []
orig_loss = []

# find index of input-target pair in test_enc that predicts TIMESTEP
dataset_id = find_target_index_in_dataset(nested_list=target_idx, target_id=TIMESTEP)

with pt.no_grad():
    inputs, targets = test_enc[dataset_id]
    # add batch dimension with unsqueeze(0)
    inputs = inputs.flatten().unsqueeze(0).to(device)
    targets = targets.unsqueeze(0).to(device)

    for step in range(PRED_HORIZON):
        # shift input sequence by one: add last prediction while discarding first input
        if step != 0:
            inputs = shift_input_sequence(orig_seq=inputs, new_pred=pred)

        # time-evolution (autoregressive)
        pred = FC_model(inputs)
        latent_loss.append(mse_loss(targets[:, :, 0], pred))

        # re-scaling
        pred_rescaled = latent_scaler.rescale(pred)

        # decoding to full space
        pred_orig = decoder(pred_rescaled.unsqueeze(0)).squeeze().detach()
        orig_loss.append(mse_loss(test[:, :, target_idx[dataset_id][step]], pred_orig))

MSE = (test[:, :, TIMESTEP] - pred_orig)**2

#### Plot Latent vs Original loss over the predictions

In [ ]:
fig = plt.subplots(1, 1, figsize=config.standard_figsize_1)
plt.plot(range(1, PRED_HORIZON + 1), latent_loss, label="Latent Loss")
plt.plot(range(1, PRED_HORIZON + 1), orig_loss, label="Orig Loss")
plt.ylabel("MSE")
plt.xlabel("number of autoregressive predictions")
plt.yscale("log")
plt.legend()
plt.tight_layout
plt.savefig(join(Path(FC_PATH).parent, "VAE_FC_predhor2_origvslatentloss.png"), bbox_inches="tight")

#### Plot Original vs predicted Snapshot

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
vmin_cp, vmax_cp = config.plot_lims_cp
vmin_MSE, vmax_MSE = config.plot_lims_MSE_reconstruction
levels_cp = pt.linspace(vmin_cp, vmax_cp, 120)
levels_MSE = pt.linspace(vmin_MSE, vmax_MSE, 120)

ax1.contourf(xx, yy, test[:, :, TIMESTEP], vmin=vmin_cp, vmax=vmax_cp, levels=levels_cp)
ax2.contourf(xx, yy, pred_orig, vmin=vmin_cp, vmax=vmax_cp, levels=levels_cp)
cont = ax3.contourf(xx, yy, MSE, vmin=vmin_MSE, vmax=vmax_MSE, levels=levels_MSE)

ax1.set_title("Ground Truth")
ax2.set_title("CNN-VAE-FC")

fig.subplots_adjust(right=0.95)
cax = fig.add_axes([0.99, 0.283, 0.03, 0.424])
cbar = fig.colorbar(cont, cax=cax,label = "Squarred Error")
cbar.formatter = ticker.FormatStrFormatter(f'%.{2}f')

for ax in [ax1, ax2, ax3]:
    ax.set_aspect("equal")
    ax.set_xticklabels([])
    ax.set_yticklabels([])

fig.savefig(join(Path(FC_PATH).parent, "VAE_FC_predhor2_timestep_reconstr.png"), bbox_inches="tight")

#### Loss vs. Prediction Horizon

In [ ]:
# show how the loss of the selected model configuration changes when the prediction horizon increases